In [66]:
import pandas as pd

def load(p: str) -> pd.DataFrame:
    df = pd.read_csv(p)
    df['transaction_reference_id'] = df['transaction_reference_id'].apply(lambda x: int(x, 16))
    return df

account_booking_test_ = load("account_booking_test.csv")
account_booking_train_ = load("account_booking_train.csv")
external_parties_test_ = load("external_parties_test.csv")
external_parties_train_ = load("external_parties_train.csv")

In [67]:
def clean_data(line):
    return line.lower() \
        .replace('mr', '') \
        .replace('ms', '') \
        .replace('mrs', '') \
        .replace('miss ', '') \
        .replace('dr', '') \
        .replace('prof', '') \
        .replace('rev', '') \
        .replace('hon', '') \
        .replace('.', '')

def clean_data_phone(line):
    if type(line) == str and line != "nan":
        return line.lower().replace('+', '').replace('-', '').replace('(', '').replace(')', '').replace(' ', '').replace(',', '').replace('.', '')
    else:
        return None
    
def clean_data_iban(line):
    if type(line) == str and line != "nan":
        return line
    else:
        return None

def cleanup(df: pd.DataFrame):
    df["party_iban"] = df["party_iban"].apply(clean_data_iban)
    df["parsed_name"] = df["parsed_name"].apply(clean_data)
    df["party_phone"] = df["party_phone"].apply(clean_data_phone)

cleanup(external_parties_train_)
cleanup(external_parties_test_)

In [68]:
# if training d
if False:
    account_booking = account_booking_train_
    external_parties = external_parties_train_
else:
    account_booking = account_booking_test_
    external_parties = external_parties_test_

In [69]:
def remove_internal(df: pd.DataFrame) -> pd.DataFrame:
    return df.drop_duplicates(subset=["transaction_reference_id"], keep=False)

account_booking = remove_internal(account_booking)

In [70]:
assert account_booking.dtypes['transaction_reference_id'] == external_parties.dtypes['transaction_reference_id']

In [71]:
joined = external_parties.set_index('transaction_reference_id').join(
    account_booking.set_index('transaction_reference_id'),
    on='transaction_reference_id'
).reset_index()

In [72]:
assert len(joined[pd.isna(joined['parsed_name'])]) == 0

In [250]:
from simhash import Simhash
from datetime import date
import pickle
from math import isnan

d_name = {}
d_city = {}

with open("d_name.pkl" , 'rb') as f:
    d_name = pickle.load(f)
with open("d_city.pkl" , 'rb') as f:
    d_city = pickle.load(f)
    
# parsed_name
# parsed_address_street_name
# parsed_address_street_number
# parsed_address_unit
# parsed_address_postal_code
# parsed_address_city
# parsed_address_state
# parsed_address_country
# party_iban
# party_phone


def bucket_iban(df: pd.DataFrame) -> int | None:
    return df['party_iban']

def bucket_street_number(df: pd.DataFrame) -> int | None:
    return df['parsed_address_street_number']

def bucket_unit(df: pd.DataFrame) -> int | None:
    return df['parsed_address_unit']

def bucket_address_state(df: pd.DataFrame) -> int | None:
    return df['parsed_address_state']

def bucket_address_country(df: pd.DataFrame) -> int | None:
    return df['parsed_address_country']

def bucket_hname(df: pd.DataFrame) -> int | None:
    # parsed_name = df['parsed_name']
    # return Simhash(parsed_name).value
    # return df['parsed_name']
    return d_name[df['transaction_reference_id']]

def bucket_name(df: pd.DataFrame) -> int | None:
    # parsed_name = df['parsed_name']
    # return Simhash(parsed_name).value
    return df['parsed_name']
    # return d_name[df['transaction_reference_id']]

def bucket_city(df: pd.DataFrame) -> int | None:
    return df['parsed_address_city']

def bucket_postal_code(df: pd.DataFrame) -> int | None:
    return df['parsed_address_postal_code']

def bucket_date(df: pd.DataFrame) -> int | None:
    return date.fromisoformat(df['transaction_date']).day

def bucket_name_street(df: pd.DataFrame) -> int | None:
    if has_field(df, 'parsed_address_street_name'):
        parsed_name = df['parsed_name']
        street = df['parsed_address_street_name']
        street = 0 if (type(street) == float and isnan(street)) else str(street)
        return f"{Simhash(parsed_name).value}-{street}"
    else:
        return f"{df['transaction_reference_id']}"
    
def has_field(df: pd.DataFrame, field: str) -> bool:
    # P => Q == not P or Q
    if field not in df:
        return False

    val = df[field]
    return val is not None and (type(val) == str or not isnan(val)) and (type(val) != str or val != "")

def bucket_name_address(df: pd.DataFrame) -> int | None:
    return f"{bucket_name(df)}-{bucket_address(df)}"

def bucket_hname_address(df: pd.DataFrame) -> int | None:
    return f"{bucket_hname(df)}-{bucket_address(df)}"

def bucket_name_paddress(df: pd.DataFrame) -> int | None:
    return f"{bucket_name(df)}-{bucket_address_partial1(df)}"

def bucket_name_state(df: pd.DataFrame) -> int | None:
    return f"{bucket_name(df)}-{bucket_address_state(df)}"

def bucket_phone(df: pd.DataFrame) -> int | None:
    if has_field(df, 'party_phone'):
        return df['party_phone']
    else:
        return df['transaction_reference_id']

def bucket_address(df: pd.DataFrame) -> int | None:
    if has_field(df, 'parsed_address_street_number') and has_field(df, 'parsed_address_street_name') and has_field(df, 'parsed_address_city') and has_field(df, 'parsed_address_postal_code'):
        num = df['parsed_address_street_number']
        street = df['parsed_address_street_name']
        city = df['parsed_address_city']
        code = df['parsed_address_postal_code']
        state = df['parsed_address_state']
        return f"{num}-{Simhash(street).value}-{city}-{state}-{code}"
    else:
        return df['transaction_reference_id']
    
def bucket_address_partial1(df: pd.DataFrame) -> int | None:
    if has_field(df, 'parsed_address_city') and has_field(df, 'parsed_address_postal_code'):
        city = df['parsed_address_city']
        code = df['parsed_address_postal_code']
        state = df['parsed_address_state']
        return f"{city}-{state}-{code}"
    else:
        return df['transaction_reference_id']


In [251]:
from typing import List, Any, Dict
from datetime import datetime

def add_linear_id(start_idx: int, df: pd.DataFrame) -> pd.DataFrame:
    df['external_id'] = range(start_idx, start_idx + len(df))
    return df

# runs the provided functions on the dataframe and progressively groups using a function
# at a time. Returns a dataframe where all rows have been grouped with an associated ID,
# and a dataframe with still ungrouped rows.
def bucket_by_fn(df: pd.DataFrame, fns: List[Any]) -> pd.DataFrame:
    already_grouped = pd.DataFrame()
    for fn in fns:
        df['bucket'] = df.apply(fn, axis=1)
        grouped_ids = df.groupby('bucket', dropna=True).filter(lambda x: len(x) > 1)
        grouped_ids['external_id'] = grouped_ids.groupby('bucket', dropna=True).ngroup() + len(df)
        new_group = grouped_ids[['transaction_reference_id', 'external_id']]
        already_grouped = pd.concat([already_grouped, new_group])
        df = df.drop(['bucket'], axis=1)
        df = df[~df['transaction_reference_id'].isin(already_grouped['transaction_reference_id'])]
        print(f"fn {fn} applied")
        
    already_grouped['transaction_reference_id'] = already_grouped['transaction_reference_id'].apply(lambda x: f"{x:0{32}x}")
    return already_grouped

In [ ]:
def do_thing(booking: pd.DataFrame, external: pd.DataFrame) -> pd.DataFrame:
    assert len(external[pd.isna(external['parsed_name'])]) == 0

    joined = external.set_index('transaction_reference_id').join(
        booking.set_index('transaction_reference_id'),
        on='transaction_reference_id'
    ).reset_index()
    print('join finished, working on', len(joined), 'elements')
    assert len(joined[pd.isna(joined['parsed_name'])]) == 0

    final = bucket_by_fn(joined, [bucket_iban, bucket_address, bucket_address_partial1, bucket_phone])
    print(f"found {len(final)} tuples")
    return final

In [ ]:
df = do_thing(account_booking_test_, external_parties_test_)
# df = do_thing(account_booking_train_, external_parties_train_)
df.to_csv('submission.csv', index=False)

join finished, working on 1481672 elements
fn <function bucket_iban at 0x7080adb6eac0> applied
fn <function bucket_name_address at 0x708034475580> applied
fn <function bucket_name_state at 0x7080344767a0> applied


In [248]:
def compare(inp: pd.DataFrame, truth: pd.DataFrame):
    tp = 0
    fp = 0
    fn = 0
    tn = 0
    
    truth_map = {}
    guess_map = {}
    for _, x in truth.iterrows():
        truth_map[x["transaction_reference_id"]] = x["external_id"]
    
    for _, x in inp.iterrows():
        guess_map[x["transaction_reference_id"]] = x["external_id"]

    inp = inp[inp.duplicated(subset='external_id', keep=False)]
    truth = truth[truth.duplicated(subset='external_id', keep=False)]

    truth_gr = truth.groupby("external_id")["transaction_reference_id"].apply(list)
    guess_gr = inp.groupby("external_id")["transaction_reference_id"].apply(list)

    for t_group in truth_gr:
        for a in t_group:
            for b in t_group:
                if a == b:
                    continue

                ag, bg = guess_map.get(a), guess_map.get(b)

                if (ag and bg) and ag == bg:
                    tp += 1
                else:
                    fn += 1

    for g_group in guess_gr:
        for a in g_group:
            for b in g_group:
                if a == b:
                    continue
                
                at, bt = truth_map[a], truth_map[b]

                if at != bt:
                    fp += 1
                else:
                    tn += 1
    
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

removed_singleton = external_parties_train_[external_parties_train_.duplicated(subset='external_id', keep=False)]
print("Number of entities: ", len(df), "/", len(removed_singleton))
print("Number of groups: ", len(df.groupby('external_id').count()), "/", len(removed_singleton.groupby('external_id').count()))
# result = do_thing(account_booking_train_, external_parties_train_)
df['transaction_reference_id'] = df['transaction_reference_id'].apply(lambda x: int(x, 16))
print(compare(df, external_parties_train_))

Number of entities:  6428 / 8093
Number of groups:  2017 / 2029
0.6615760266370699


In [100]:
groups = df.set_index('transaction_reference_id').join(
        external_parties_train_.drop('external_id', axis=1).set_index('transaction_reference_id'),
        on='transaction_reference_id'
    ).reset_index().groupby('external_id')

g = pd.DataFrame()
for i, grp in groups:
    g = pd.concat([g, grp])
    if i > 4:
        break

g

,transaction_reference_id,external_id,party_role,party_info_unstructured,parsed_name,parsed_address_street_name,parsed_address_street_number,parsed_address_unit,parsed_address_postal_code,parsed_address_city,parsed_address_state,parsed_address_country,party_iban,party_phone
4756,329607983514487012669631641585391708093,6342,BENE,b. johnson 177 sherry corner haleyfort,b johnson,sherry corner,177,NaN,NaN,haleyfort,NaN,NaN,None,004100127881000747212
4790,101892866522542013383394373256733168479,6342,BENE,bradley johnson 177 sherry corner haleyfort,bradley johnson,sherry corner,177,NaN,NaN,haleyfort,NaN,NaN,None,004100127881000747212


In [ ]:
result = do_thing(account_booking_train_, external_parties_train_)
df = pd.DataFrame(list(result.items()), columns=['transaction_reference_id', 'external_id'])
df.to_csv('submission.csv', index=False)

join finished, working on 11064 elements
fn <function bucket_iban at 0x7ff7b1508040> applied
found 3872 tuples


In [ ]:
booking = pd.DataFrame(account_booking_train_).drop_duplicates(subset=["transaction_reference_id"], keep=False)
parties = external_parties_train_[external_parties_train_.duplicated(subset="external_id", keep=False)]
data = booking.merge(parties, on="transaction_reference_id", how="inner").groupby('external_id').agg(list).reset_index()
print(data.iloc[0])

external_id                                                              20000042
transaction_reference_id        [313980040502377422398921352947482856636, 1670...
debit_credit_indicator                                   [CREDIT, CREDIT, CREDIT]
account_id                                                  [28523, 25210, 21675]
transaction_amount                                    [8101.48, 5879.79, 4590.15]
transaction_currency                                              [GBP, GBP, GBP]
transaction_date                             [2023-07-10, 2023-06-02, 2024-10-24]
party_role                                                        [ORG, ORG, ORG]
party_info_unstructured         [m. solomon 08/04/2000 826 gwendolyn plaza apt...
parsed_name                         [m solomon, marsolomon, mary mary solomon ii]
parsed_address_street_name      [gwendolyn plaza, gwendolyn plaza apt., gwendo...
parsed_address_street_number                                      [826, 826, 826]
parsed_address_u